In [8]:
#setup
import warnings
from typing import List, Optional, Union
from datetime import datetime, date
from dateutil.parser import parse
from watttime import api, report
from itertools import product

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', message="Found version mismatch between `dagster`")

In [9]:
#input parameters
ba_list: List[str] = ["WEM"]
model_date_list: List[str] = ["2022-10-01", "2024-10-16"]
signal_type: str = "co2_moer"
eval_start: Union[str, datetime, date] = '2024-01-01T00:00Z'
eval_end: Union[str, datetime, date] = '2025-01-01T00:00Z'

api_username: Optional[str] = None  # if None, will look for env var WATTTIME_USER
api_password: Optional[str] = None  # if None, will look for env var WATTTIME_PASSWORD

forecast_sample_days: int = 5

In [10]:
WT_FORECAST = api.WattTimeForecast(username=api_username, password=api_password)
WT_HIST = api.WattTimeHistorical(username=api_username, password=api_password)

EVAL_DAYS = report.parse_eval_days(eval_start=eval_start, eval_end=eval_end)
SAMPLE_DAYS = report.parse_forecast_sample_days(eval_days=EVAL_DAYS, forecast_sample_days=forecast_sample_days)

In [11]:
jobs = list(
    report.ModelAnalysis(
        ba=i[0],
        model_date=i[1],
        signal_type=signal_type,
        eval_start=eval_start, eval_end=eval_end,
        eval_days=EVAL_DAYS, sample_days=SAMPLE_DAYS,
        wt_forecast=WT_FORECAST, wt_hist=WT_HIST
    ) for i in product(ba_list, model_date_list)
)

# pull all required data and compile
for job in jobs:
    job.compile_forecast_v_moer()


 Warnings Returned: {'region': 'WEM', 'signal_type': 'co2_moer', 'model': '2022-10-01', 'start': datetime.datetime(2024, 7, 29, 0, 0, tzinfo=<UTC>), 'end': datetime.datetime(2024, 8, 27, 23, 55, tzinfo=<UTC>)} {'data_point_period_seconds': 300, 'region': 'WEM', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MISSING_DATA', 'message': 'You made a valid request, but we have 560 records missing. This can happen for a variety of reasons. If the data is less than a week old, we will most likely fill in this data soon. If it is older than that, we are missing enough important grid data during this time period that we are not able to generate an accurate estimate'}], 'model': {'date': '2022-10-01', 'type': 'binned_regression'}}

 Warnings Returned: {'region': 'WEM', 'signal_type': 'co2_moer', 'model': '2022-10-01', 'start': datetime.datetime(2024, 9, 27, 0, 0, tzinfo=<UTC>), 'end': datetime.datetime(2024, 10, 26, 23, 55, tzinfo=<UTC>)} {'data_point_period_second

## Data Distribution Changes


Below is a summary of how you can expect signal values to change between versions.

In [12]:
report.plot_sample_moers(jobs, max_sample_period='7D')

In [13]:
report.plot_distribution_moers(jobs)

In [14]:
report.plot_ba_heatmaps(jobs)

## Forecast Performance

We assess the overall performance of our forecasts using a few main metrics, described below.


* **Normalized Mean Absolute Error (Norm MAE):** Mean Absolute Error (MAE) is the average of absolute forecast errors. This represents the magnitude of error to expect from the forecast on average. A model with an MAE closest to 0 is ideal. Norm MAE is MAE divided by the average of the true values. Normalizing MAE allows us to compare models across different datasets. It is expressed as a percentage (%) of the average true value over the validation time period. 
* **Rank Correlation:** A number between -1 and 1 that quantifies the degree to which the rank ordering of two datasets are related. A rank correlation of 1 means the forecast predictions are rank ordered perfectly compared to the true values, making the forecast perfectly effective for time optimization use cases. For many use cases of our forecasts models (such as AER), it is more important that predictions are correctly rank ordered within a forecast window, rather than each forecast being accurate in terms of absolute values.
* **AER CO2 Savings:** This metric represents the impact in lbs CO2/MWh of using the forecast model for energy usage optimization. We compare these CO2 savings against the "Max Potential" case (ie. the case of using perfect forecasts). In this notebook, we model 3 different popular AER Scenarios:
  
  1. EV-night: Charging an electric vehicle for 3 hours out of a 12-hour overnight window starting at 19:00 local time.
  2. EV-day: Charging an electric vehicle for 2 hours out of an 8-hour daytime window starting at 09:00 local time.
  3. Thermostat: Optimizing thermostat usage in continual 1-hour windows throughout a 24-hour period, only actively using energy for half of the time. 



In [15]:
report.plot_norm_mae(jobs)

In [16]:
report.plot_rank_corr(jobs)

In [ ]:
report.plot_impact_forecast_metrics(jobs)